# 공유 Qdrant에 어떤 collections들 있는지 확인용

## 리스트 
(밑에 셀에 다 삭제하는 코드 있음)

In [1]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient

# Load Qdrant credentials from the project .env file
load_dotenv("../.env")

client = QdrantClient(
    url=os.environ["QDRANT_URL"],
    api_key=os.environ["QDRANT_API_KEY"],
)

collections_resp = client.get_collections()

total_cnt = 0
for coll in collections_resp.collections:
    total_cnt += 1
    info = client.get_collection(coll.name)  # CollectionInfo
    vectors = info.vectors_count or {}
    total_vectors = sum(v.count for v in vectors.values()) if vectors else "n/a"

    print(
        f"- {coll.name} | points: {info.points_count} | vectors: {total_vectors}"
    )

print(f"Total # of collections: {total_cnt}")

- trade_collectiont | points: 3696 | vectors: n/a
Total # of collections: 1


## 삭제용

In [4]:
def delete_all_collections():
    """Delete all collections in Qdrant."""
    collections_resp = client.get_collections()
    
    deleted_count = 0
    for coll in collections_resp.collections:
        try:
            client.delete_collection(coll.name)
            print(f"✓ Deleted collection: {coll.name}")
            deleted_count += 1
        except Exception as e:
            print(f"✗ Failed to delete {coll.name}: {e}")
    
    print(f"\nTotal collections deleted: {deleted_count}")


def delete_target_collection(collection_name: str):
    """
    Delete a specific collection in Qdrant.
    
    Args:
        collection_name: Name of the collection to delete
    """
    try:
        client.delete_collection(collection_name)
        print(f"✓ Successfully deleted collection: {collection_name}")
    except Exception as e:
        print(f"✗ Failed to delete collection '{collection_name}': {e}")

### 원하는 거 하나만 지우기

In [16]:
delete_target_collection("cisg_document_collection")

✓ Successfully deleted collection: cisg_document_collection


### 전부 지우기

In [14]:
delete_all_collections()  # ⚠️ WARNING: This will delete ALL collections!

✓ Deleted collection: trade_collection

Total collections deleted: 1


fraud 데이터 이상한 point/chunk 삭제

In [1]:
# 방법 2: data_source 필드가 없거나 'fraud'인 포인트 모두 삭제

from qdrant_client import QdrantClient
import os
from dotenv import load_dotenv

load_dotenv()

COLLECTION_NAME = "trade_collection"

qdrant = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY"),
    timeout=300
)

print("fraud 관련 포인트 검색 중...")

# 전략: 
# 1. 모든 포인트를 스캔 (scroll API 사용)
# 2. data_source가 없거나 'fraud'인 포인트 ID 수집
# 3. 해당 ID들 삭제

all_points_to_delete = []
offset = None

while True:
    result = qdrant.scroll(
        collection_name=COLLECTION_NAME,
        limit=100,
        offset=offset,
        with_payload=True
    )
    
    points, offset = result
    
    for point in points:
        payload = point.payload or {}
        data_source = payload.get("data_source")
        
        # data_source가 없거나 'fraud'인 경우
        if data_source is None or data_source == 'fraud':
            all_points_to_delete.append(point.id)
    
    if offset is None:
        break

print(f"삭제할 포인트 수: {len(all_points_to_delete)}")

if all_points_to_delete:
    # 배치로 삭제
    batch_size = 100
    for i in range(0, len(all_points_to_delete), batch_size):
        batch = all_points_to_delete[i:i+batch_size]
        qdrant.delete(
            collection_name=COLLECTION_NAME,
            points_selector=batch
        )
        print(f"삭제 진행: {i+len(batch)}/{len(all_points_to_delete)}")
    
    print(f"✓ {len(all_points_to_delete)}개 포인트 삭제 완료")
else:
    print("삭제할 포인트가 없습니다.")

# 삭제 후 확인
collection_info = qdrant.get_collection(COLLECTION_NAME)
print(f"\n현재 총 포인트 수: {collection_info.points_count}")

fraud 관련 포인트 검색 중...
삭제할 포인트 수: 154
삭제 진행: 100/154
삭제 진행: 154/154
✓ 154개 포인트 삭제 완료

현재 총 포인트 수: 1375
